In [ ]:
## Package Import
import sys
import os 
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
## Setting for running dsub jobs
pd.set_option('display.max_colwidth', 0)

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env USER_NAME={USER_NAME}

In [ ]:
## MODIFY FOR FULL DATA RUN
# Use hyphens, not whitespace since it will become part of the bucket path.
## TODO: add job name
JOB_NAME=''

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env JOB_NAME={JOB_NAME}

In [ ]:
## Analysis Results Folder
line_count_results_folder = os.path.join(
    os.getenv('WORKSPACE_BUCKET'),
    'dsub',
    'results',
    JOB_NAME,
    USER_NAME,
    datetime.now().strftime('%Y%m%d'))

line_count_results_folder

In [ ]:
## Where the output files will go
output_files = os.path.join(line_count_results_folder, "results")
print(output_files)

In [ ]:
OUTPUT_FILES = output_files


# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env OUTPUT_FILES={OUTPUT_FILES}

## Print reads and read depth into one step

In [ ]:
Can be modified for bams

In [ ]:
%%writefile ~/printreads_depth.sh

set -o pipefail
set -o errexit


# ---------Required Inputs---------
# aou_crams - A .txt file containing gs:// paths to cram samples.

# Given a .txt file - get X samples.
# For parallel submissions:
# - Use a different .txt file per submission.
# - Each .txt file can contain a different number of lines
aou_crams_len=$(wc -l < ${aou_crams})
echo "Samples in cramlist: ${aou_crams_len}"

# ---------Required Output---------
#filtered_cram_output

echo "GOOGLE_PROJECT: ${GOOGLE_PROJECT}"
echo "OUTPUT_PATH: ${OUTPUT_PATH}"
echo "ref_dict: ${ref_dict}"
echo "ref_fai: ${ref_fai}"
echo "ref_fasta: ${ref_fasta}"

# Perform runs for x samples.
for (( i=1; i<$aou_crams_len+1; i++ ));
do

    # These change per iteration
    export aou_cram_reads=$(sed "${i}!d;q" "${aou_crams}")   # gs:// path to a cram sample
    export aou_cram_reads_name=`basename ${aou_cram_reads}`  # file_name.cram
    export aou_cram_reads_prefix="${aou_cram_reads_name%.*}" # file_name
    echo "aou_cram_reads: ${aou_cram_reads}"
    echo "aou_cram_reads_name: ${aou_cram_reads_name}"
    echo "aou_cram_reads_prefix: ${aou_cram_reads_prefix}"

    # ----------------------------------WORKFLOW----------------------------------

    # Stream CRAM found at gs:// path into a new, smaller CRAM based on genomic intervals given with -L.
    /gatk/gatk PrintReads \
        -I ${aou_cram_reads} \
        -L "chr14:21621904-22552132" \
        -R "${ref_fasta}" \
        -O "${aou_cram_reads_prefix}_tcr.cram" \
        --gcs-project-for-requester-pays ${GOOGLE_PROJECT} \
        --cloud-prefetch-buffer 0 --cloud-index-prefetch-buffer 0

    # Create CRAI index file for the new CRAM.
    samtools index "${aou_cram_reads_prefix}_tcr.cram" "${aou_cram_reads_prefix}_tcr.crai"
    
    samtools depth -a --reference ${ref_fasta} -r chr14:21621904-22552132 "${aou_cram_reads_prefix}_tcr.cram" > "${aou_cram_reads_prefix}_tcr.txt"


    # Outputs
    export tcr_depth_tsv="${aou_cram_reads_prefix}_tcr.txt"
    echo "tcr_depth_tsv: ${tcr_depth_tsv}"

    # Disk space
    echo "Disk space taken up so far:"
    du -d 1 -h
    echo "${i} run(s) finished."

    # Move results to output directory
    mv ${tcr_depth_tsv} ${OUTPUT_PATH}
done

In [ ]:
%%bash --out LINE_COUNT_JOB_ID

# Get a shorter username to leave more characters for the job name.
DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

# For AoU RWB projects network name is "network".
AOU_NETWORK=network
AOU_SUBNETWORK=subnetwork

# Get all cramlists
bashArray=()

while read line; do
  bashArray+=($line)
done < ## TODO add pathway to bam/cram pathway

# Length of entire array
len_bashArray=${#bashArray[@]}

##TODO fill in lower and upper based on bash array lenght
LOWER=
UPPER=
DATE=
MACHINE_TYPE="n2-standard-4"
BASH_SCRIPT="" ## TODO add bash script pathway
for ((batch=$LOWER;batch<$UPPER;batch+=1))
do
    dsub \
        --provider google-cls-v2 \
        --user-project "${GOOGLE_PROJECT}"\
        --project "${GOOGLE_PROJECT}"\
        --network "${AOU_NETWORK}" \
        --subnetwork "${AOU_SUBNETWORK}" \
        --service-account "$(gcloud config get-value account)" \
        --user "${DSUB_USER_NAME}" \
        --regions us-central1 \
        --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/${DATE}/${bashArray[batch]}.log" \
        "$@" \
        --preemptible \
        --boot-disk-size 100 \
        --machine-type ${MACHINE_TYPE} \
        --disk-size 100 \
        --name "${JOB_NAME}" \
        --script "${BASH_SCRIPT}" \
        --image 'gcr.io/bick-aps2/briansha/pileup_preprocessing:latest' \
        --env GOOGLE_PROJECT=${GOOGLE_PROJECT} \
        --input ref_dict="" \ ## TODO add reference dictionary pathway
        --input ref_fai="" \ ## TODO add reference fasta index pathway
        --input ref_fasta="" \ ## TODO add reference fasta pathway
        --input aou_crams="${bashArray[batch]}" \
        --output-recursive OUTPUT_PATH="${OUTPUT_FILES}/${batch}"
done